In [ ]:
!wget 'https://hikari.butaishoujo.moe/t/3484b4d4/Aqours%20-%20PV%20-%20HAPPY%20PARTY%20TRAIN.txt'
!wget 'https://hikari.butaishoujo.moe/v/71921534/Aqours%20-%20PV%20-%20HAPPY%20PARTY%20TRAIN.mp4'


In [ ]:
import logging
from pathlib import Path

import torchaudio

from yohane.audio import VocalRemoverVocalsExtractor, compute_alignments
from yohane.lyrics import Lyrics
from yohane.subtitles import make_ass

logging.basicConfig(level="INFO")
logger = logging.getLogger(__name__)


In [ ]:
SONG_FILE = Path('Aqours - PV - HAPPY PARTY TRAIN.mp4')
VOCALS_EXTRACTOR_CLS = VocalRemoverVocalsExtractor

waveform, sample_rate = torchaudio.load(SONG_FILE)  # type: ignore

if VOCALS_EXTRACTOR_CLS is not None:
    extractor = VOCALS_EXTRACTOR_CLS()
    logger.info(f"Extracting vocals with {extractor=}")
    waveform, sample_rate = extractor(waveform, sample_rate)
    torchaudio.save(SONG_FILE.with_suffix(".vocals.wav"), waveform, sample_rate)  # type: ignore


In [ ]:
LYRICS_FILE = Path('Aqours - PV - HAPPY PARTY TRAIN.txt')

with open(LYRICS_FILE) as fp:
    lyrics = Lyrics(fp.read())


In [ ]:
emission, token_spans = compute_alignments(waveform, sample_rate, lyrics.transcript)


In [ ]:
subs = make_ass(lyrics, waveform, sample_rate, emission, token_spans)

print(subs.to_string('ass'))

subs_file = SONG_FILE.with_suffix(".ass")
subs.save(subs_file.as_posix())
